## 섹터 0 & 1: 기본 설정 (환경, 경로, 하이퍼파라미터)

In [ ]:
# ================================================================
# [섹터 0 & 1] 환경 준비 및 기본 설정 (오류 수정)
# ================================================================

# --- 환경 설정 ---
from google.colab import drive
drive.mount('/content/drive')

# --- 기본 라이브러리 임포트 ---
import torch  # <--- 이 줄이 추가되었습니다.
import os, json, random, math
from pathlib import Path
from collections import defaultdict
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True # 손상된 이미지도 최대한 로드

# --- 경로 설정 ---
# 1. 프로젝트 최상위 폴더 (수정 필요 시)
ROOT = Path("/content/drive/MyDrive/datasets/pills")

# 2. 하위 폴더 및 파일 경로 (자동 설정)
IMG_DIR = ROOT / "train_images"
ANN_DIR = ROOT / "train_annotations"      # 원본 Annotation 폴더
OUTPUT_DIR = ROOT / "outputs"             # EDA 결과물이 저장된 폴더

# EDA에서 생성한 데이터 분할 파일을 사용하도록 경로 변경
SPLITS_PATH = OUTPUT_DIR / "RetinaNet_splits.json"
MERGED_JSON = OUTPUT_DIR / "RetinaNet_coco_merged.json" # 통합 Annotation 파일 저장 경로
CKPT_DIR = ROOT / "checkpoints"
CKPT_DIR.mkdir(parents=True, exist_ok=True)

# --- 하이퍼파라미터 ---
SEED = 42
BATCH_SIZE = 4      # GPU 메모리 부족(OOM) 시 2로 줄이기
NUM_WORKERS = 2     # Colab 기본값
EPOCHS = 50         # 전체 학습 에폭 수
LR = 0.005          # 학습률 (Learning Rate)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_AMP = True      # 혼합 정밀도 사용 (메모리 절약, 속도 향상)

# 시드 고정 (실험 재현성 확보)
random.seed(SEED)
torch.manual_seed(SEED)

# --- 경로 존재 여부 확인 ---
assert IMG_DIR.exists(), f"이미지 폴더가 없습니다: {IMG_DIR}"
assert ANN_DIR.exists(), f"Annotation 폴더가 없습니다: {ANN_DIR}"
print(f"\n[INFO] 프로젝트 경로: {ROOT}")
print(f"[INFO] 학습 장치: {DEVICE}")

Mounted at /content/drive

[INFO] 프로젝트 경로: /content/drive/MyDrive/datasets/pills
[INFO] 학습 장치: cuda


## 섹터 2: Annotation 파일 병합

In [2]:
# ================================================================
# [섹터 2] Annotation 파일 병합 (없을 경우에만 실행) - (수정)
# ================================================================
# 설명: 여러 JSON 파일을 하나로 합치면서, 모든 이미지와 Annotation에
#      고유한 ID를 새로 부여하여 데이터 충돌을 방지합니다.

FORCE_REBUILD = False # True로 설정 시 항상 새로 병합

if MERGED_JSON.exists() and not FORCE_REBUILD:
    with open(MERGED_JSON, "r", encoding="utf-8") as f:
        COCO = json.load(f)
    print(f"[INFO] 캐시된 통합 Annotation 파일 사용: {MERGED_JSON}")
else:
    print(f"[INFO] 통합 Annotation 파일({MERGED_JSON})이 없어 새로 생성합니다.")

    # --- (추가) 실제 파일 병합 로직 ---
    # 원본 Annotation 폴더에서 모든 .json 파일을 찾음
    json_files = [p for p in ANN_DIR.rglob("*.json")]
    assert json_files, f"Annotation 파일을 찾을 수 없습니다: {ANN_DIR}"

    # 병합된 데이터를 담을 컨테이너
    new_images = []
    new_annotations = []
    category_names = set() # 중복 없는 카테고리 이름을 저장

    # ID를 새로 부여하기 위한 카운터
    next_img_id = 1
    next_ann_id = 1

    for json_path in json_files:
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
        except Exception as e:
            print(f"경고: {json_path} 파일 읽기 실패. 건너뜁니다. ({e})")
            continue

        # 각 파일의 이미지 ID를 전역 ID로 매핑하기 위한 딕셔너리
        old_to_new_img_id = {}

        # 이미지 정보 병합
        for img in data.get('images', []):
            file_name = Path(img['file_name']).name
            new_img = {
                'id': next_img_id,
                'file_name': file_name,
                'width': img['width'],
                'height': img['height']
            }
            new_images.append(new_img)
            old_to_new_img_id[img['id']] = next_img_id
            next_img_id += 1

        # 카테고리 이름 수집
        local_cats = {c['id']: c['name'] for c in data.get('categories', [])}
        for name in local_cats.values():
            category_names.add(name)

        # Annotation 정보 병합
        for ann in data.get('annotations', []):
            # 유효하지 않은 Annotation은 건너뜀
            if 'bbox' not in ann or ann['image_id'] not in old_to_new_img_id:
                continue

            x, y, w, h = ann['bbox']
            if not (w > 0 and h > 0):
                continue

            new_ann = {
                'id': next_ann_id,
                'image_id': old_to_new_img_id[ann['image_id']],
                'category_name': local_cats.get(ann['category_id']), # 임시로 이름 저장
                'bbox': ann['bbox'],
                'iscrowd': ann.get('iscrowd', 0),
                'area': w * h
            }
            new_annotations.append(new_ann)
            next_ann_id += 1

    # 전체 카테고리 이름에 대해 고유 ID 부여
    sorted_names = sorted(list(category_names))
    name_to_cat_id = {name: i + 1 for i, name in enumerate(sorted_names)}

    new_categories = [{'id': cat_id, 'name': name} for name, cat_id in name_to_cat_id.items()]

    # Annotation에 최종 category_id 할당
    for ann in new_annotations:
        ann['category_id'] = name_to_cat_id[ann['category_name']]
        del ann['category_name'] # 임시 키 제거

    # 최종 COCO 객체 생성
    COCO = {
        "images": new_images,
        "annotations": new_annotations,
        "categories": new_categories
    }

    # 파일로 저장하여 다음 실행 시 재사용
    OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
    with open(MERGED_JSON, "w", encoding="utf-8") as f:
        json.dump(COCO, f, ensure_ascii=False, indent=2)

    print(f"[SUCCESS] 파일 병합 완료 및 저장: {MERGED_JSON}")
    print(f"  -> 이미지: {len(COCO['images'])}개, Annotation: {len(COCO['annotations'])}개, 클래스: {len(COCO['categories'])}개")

[INFO] 캐시된 통합 Annotation 파일 사용: /content/drive/MyDrive/datasets/pills/outputs/RetinaNet_coco_merged.json


## 섹터 3 & 4: 인덱싱 및 데이터 분할

In [3]:
# ================================================================
# [섹터 3 & 4] 인덱싱 및 데이터 분할
# ================================================================

# --- 섹터 3: 인덱스 생성 ---
img_meta = {im["id"]: im for im in COCO["images"]}
ann_by_img = defaultdict(list)
for a in COCO["annotations"]:
    ann_by_img[a["image_id"]].append(a)
id2path = {i: (IMG_DIR / Path(img_meta[i]["file_name"]).name) for i in img_meta}
K = len(COCO["categories"])
print(f"[INFO] 인덱싱 완료: 이미지 {len(img_meta)}개, 클래스 {K}개")


# --- 섹터 4: 데이터 분할 ---
# 설명: EDA 단계에서 생성된 RetinaNet_splits.json 파일을 읽어
#      학습용(train_ids)과 검증용(val_ids) 이미지 ID 리스트를 만듭니다.
assert SPLITS_PATH.exists(), f"데이터 분할 파일이 없습니다: {SPLITS_PATH}"

with open(SPLITS_PATH, "r", encoding="utf-8") as f:
    splits = json.load(f)

train_ids = splits['train_ids']
val_ids = splits['val_ids']

# 실제 파일이 존재하는 ID만 최종 사용
train_ids = [i for i in train_ids if id2path[i].exists()]
val_ids = [i for i in val_ids if id2path[i].exists()]

assert len(train_ids) > 0 and len(val_ids) > 0, "학습 또는 검증 데이터가 없습니다."
print(f"[INFO] 데이터 분할 완료: Train {len(train_ids)}개 / Validation {len(val_ids)}개")

[INFO] 인덱싱 완료: 이미지 4526개, 클래스 73개
[INFO] 데이터 분할 완료: Train 4060개 / Validation 466개


## 섹터 5 & 6: 데이터셋 클래스 및 전처리

In [4]:
# ================================================================
# [섹터 5 & 6] 전처리 함수 및 Dataset 클래스 정의
# ================================================================
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset
from torchvision.transforms.functional import to_tensor

# --- 섹터 5: 전처리/증강 함수 ---
# (이전 답변과 동일. xywh_to_xyxy_and_clip, resize_keep_aspect, hflip_boxes_xyxy)
def xywh_to_xyxy_and_clip(bbox, W, H):
    x, y, w, h = bbox; x1=max(0.0,x); y1=max(0.0,y); x2=min(float(W),x+w); y2=min(float(H),y+h); return [x1,y1,x2,y2]
def resize_keep_aspect(img, boxes, short=800, max_sz=1333):
    W,H=img.size; s,l=(H,W) if H<W else (W,H); sc=short/max(1,s)
    if round(l*sc)>max_sz: sc=max_sz/max(1,l)
    if abs(sc-1.0)<1e-6: return img, boxes
    nW,nH=int(round(W*sc)), int(round(H*sc)); img=img.resize((nW,nH),Image.BILINEAR)
    if boxes.numel()>0: boxes=boxes*sc
    return img, boxes
def hflip_boxes_xyxy(boxes, w):
    if boxes.numel()==0: return boxes
    x1=boxes[:,0].clone(); x2=boxes[:,2].clone(); boxes[:,0]=w-x2; boxes[:,2]=w-x1
    return boxes

# --- 섹터 6: Dataset 클래스 정의 ---
class CocoLikeDetection(Dataset):
    def __init__(self, image_ids: list, augment: bool = True):
        self.ids = image_ids
        self.augment = augment

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        im_id = self.ids[idx]
        with Image.open(id2path[im_id]) as im:
            im = im.convert("RGB")
            W, H = im.size

        # Annotation 로드 및 유효한 Bbox만 필터링
        anns = ann_by_img.get(im_id, [])
        boxes_xyxy, labels = [], []
        for a in anns:
            x1, y1, x2, y2 = xywh_to_xyxy_and_clip(a["bbox"], W, H)
            if x2 > x1 and y2 > y1: # 너비와 높이가 0보다 큰 유효한 박스만 추가
                boxes_xyxy.append([x1, y1, x2, y2])
                labels.append(int(a["category_id"])) # 원본 ID는 1...K

        boxes = torch.tensor(boxes_xyxy, dtype=torch.float32) if boxes_xyxy else torch.zeros((0, 4))
        labels = torch.tensor(labels, dtype=torch.int64) if labels else torch.zeros((0,))

        # RetinaNet은 0-based label을 사용 (0 ~ K-1)
        if labels.numel():
            labels = labels - 1

        # 전처리 및 증강
        im, boxes = resize_keep_aspect(im, boxes)
        if self.augment:
            if random.random() < 0.5:
                im = TF.hflip(im)
                boxes = hflip_boxes_xyxy(boxes, im.size[0])
            im = TF.adjust_brightness(im, 0.9 + 0.2 * random.random())
            im = TF.adjust_contrast(im, 0.9 + 0.2 * random.random())

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([im_id])
        }
        return to_tensor(im), target

def collate(batch):
    return list(zip(*batch))

print("Dataset 클래스 정의 완료.")

Dataset 클래스 정의 완료.


## 섹터 7 & 8: 데이터로더 및 모델 준비

In [5]:
# ================================================================
# [섹터 7 & 8] 데이터로더 및 모델 준비
# ================================================================

from torch.utils.data import DataLoader

# --- 섹터 7: DataLoader 생성 ---
train_ds = CocoLikeDetection(train_ids, augment=True)
val_ds   = CocoLikeDetection(val_ids, augment=False)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, collate_fn=collate)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, collate_fn=collate)
print(f"[INFO] DataLoader 준비 완료: Train {len(train_loader)} 배치 / Validation {len(val_loader)} 배치")

# --- 섹터 8: 모델, 옵티마이저, 스케줄러 준비 ---
# torchvision 버전에 따라 호환되는 RetinaNet을 자동으로 불러옵니다.
try:
    from torchvision.models.detection import retinanet_resnet50_fpn_v2 as retinanet_factory
except ImportError:
    from torchvision.models.detection import retinanet_resnet50_fpn as retinanet_factory

# 사전 학습된 Backbone 가중치를 사용하고, Head 부분만 우리 데이터에 맞게 초기화합니다.
model = retinanet_factory(weights_backbone="DEFAULT", num_classes=K)
model.to(DEVICE)

# 옵티마이저 (SGD) 및 학습률 스케줄러 (StepLR)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=LR, momentum=0.9, weight_decay=1e-4)
lr_sch = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# AMP(혼합 정밀도) 스케일러
scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP)

print(f"[INFO] RetinaNet 모델 준비 완료. 총 클래스 수: {K}")

[INFO] DataLoader 준비 완료: Train 1015 배치 / Validation 117 배치
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 106MB/s]


[INFO] RetinaNet 모델 준비 완료. 총 클래스 수: 73


/tmp/ipython-input-2425322579.py:31: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP)


## 섹터 9: 학습 루프

In [7]:
# ================================================================
# [섹터 9] 학습 루프 (IndexError 수정)
# ================================================================
import math
from torch.amp import autocast

# --- 수정된 학습/검증 함수 ---
def train_one_epoch(loader):
    model.train()
    total_loss = 0
    for imgs, targets in loader:
        imgs = [im.to(DEVICE) for im in imgs]
        # targets를 GPU로 보냅니다.
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        # --- (오류 수정) ---
        # 'labels' 텐서의 타입을 torch.long (int64)으로 확실하게 지정합니다.
        for t in targets:
            t['labels'] = t['labels'].to(torch.long)
        # --------------------

        optimizer.zero_grad()
        with autocast('cuda', enabled=USE_AMP):
            loss_dict = model(imgs, targets)
            loss = sum(l for l in loss_dict.values())

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
    return total_loss / len(loader)

@torch.no_grad()
def validate_loss(loader):
    model.train() # loss 계산을 위해 train 모드
    total_loss = 0
    for imgs, targets in loader:
        imgs = [im.to(DEVICE) for im in imgs]
        # targets를 GPU로 보냅니다.
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        # --- (오류 수정) ---
        # 검증 데이터셋의 'labels' 텐서 타입도 동일하게 맞춰줍니다.
        for t in targets:
            t['labels'] = t['labels'].to(torch.long)
        # --------------------

        with autocast('cuda', enabled=USE_AMP):
            loss_dict = model(imgs, targets)
            loss = sum(l for l in loss_dict.values())
        total_loss += loss.item()
    model.eval() # 다시 eval 모드로
    return total_loss / len(loader)

# --- 학습 시작 ---
best_val_loss = float('inf')
for epoch in range(1, EPOCHS + 1):
    train_loss = train_one_epoch(train_loader)
    val_loss = validate_loss(val_loader)
    lr_sch.step()

    print(f"[Epoch {epoch:02d}] Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), CKPT_DIR / "best_model.pth")
        print(f"  -> Best model saved with val_loss: {best_val_loss:.4f}")

# 가장 좋았던 모델의 가중치를 다시 불러옵니다.
print(f"\n[INFO] 학습 완료. Best Val Loss: {best_val_loss:.4f}")
model.load_state_dict(torch.load(CKPT_DIR / "best_model.pth", map_location=DEVICE))
print("[INFO] Best model weights loaded.")

[Epoch 01] Train Loss: 0.6258 | Val Loss: 0.5670
  -> Best model saved with val_loss: 0.5670
[Epoch 02] Train Loss: 0.5279 | Val Loss: 0.4847
  -> Best model saved with val_loss: 0.4847
[Epoch 03] Train Loss: 0.4599 | Val Loss: 0.4912
[Epoch 04] Train Loss: 0.4218 | Val Loss: 0.4051
  -> Best model saved with val_loss: 0.4051
[Epoch 05] Train Loss: 0.4070 | Val Loss: 0.4202
[Epoch 06] Train Loss: 0.3338 | Val Loss: 0.3441
  -> Best model saved with val_loss: 0.3441
[Epoch 07] Train Loss: 0.3224 | Val Loss: 0.3410
  -> Best model saved with val_loss: 0.3410
[Epoch 08] Train Loss: 0.3178 | Val Loss: 0.3398
  -> Best model saved with val_loss: 0.3398
[Epoch 09] Train Loss: 0.3145 | Val Loss: 0.3324
  -> Best model saved with val_loss: 0.3324
[Epoch 10] Train Loss: 0.3111 | Val Loss: 0.3347
[Epoch 11] Train Loss: 0.3037 | Val Loss: 0.3275
  -> Best model saved with val_loss: 0.3275
[Epoch 12] Train Loss: 0.3029 | Val Loss: 0.3267
  -> Best model saved with val_loss: 0.3267
[Epoch 13] Train

KeyboardInterrupt: 

## 섹터 9-1: 최고 성능 모델 불러오기

In [8]:
# ================================================================
# [섹터 9] 나머지 부분: 최고 성능 모델 불러오기
# ================================================================

# best_val_loss 변수가 없으면 임시로 0으로 설정합니다.
if 'best_val_loss' not in locals():
    best_val_loss = 0.0

# 학습 루프가 끝난 후, 가장 성능이 좋았던 모델의 가중치를 불러옵니다.
print(f"\n[INFO] 학습 중단/완료. 마지막 Best Val Loss: {best_val_loss:.4f}")

# 저장된 best_model.pth 파일의 경로
best_model_path = CKPT_DIR / "best_model.pth"

# 파일이 존재하는지 확인 후 모델에 가중치를 로드
if best_model_path.exists():
    model.load_state_dict(torch.load(best_model_path, map_location=DEVICE))
    print(f"[INFO] 최고 성능 모델({best_model_path.name})을 성공적으로 불러왔습니다.")
else:
    print(f"[ERROR] best_model.pth 파일을 찾을 수 없습니다! 학습이 정상적으로 진행되었는지 확인해주세요.")


[INFO] 학습 중단/완료. 마지막 Best Val Loss: 0.3250
[INFO] 최고 성능 모델(best_model.pth)을 성공적으로 불러왔습니다.


## 섹터 10: 평가 (mAP) 및 제출 파일 생성

In [11]:
# ================================================================
# [섹터 10] 평가(mAP) 및 제출 파일 생성 (KeyError 수정)
# ================================================================
# pycocotools가 설치되어 있는지 확인, 없으면 설치
try:
    from pycocotools.coco import COCO as COCOapi
    from pycocotools.cocoeval import COCOeval
except ImportError:
    print("pycocotools 설치를 시작합니다...")
    !pip install -q pycocotools
    from pycocotools.coco import COCO as COCOapi
    from pycocotools.cocoeval import COCOeval

import csv
import numpy as np
from torchvision.transforms.functional import to_tensor

# --- 평가 및 제출용 설정값 ---
TEST_DIR = ROOT / "test_images"
SUBMIT_CSV = ROOT / "submission.csv"
EVAL_SCORE_THR = 0.05  # mAP 평가 시 예측으로 인정할 최소 점수(confidence)

# --- 함수 정의 ---
def build_gt_for_eval(val_ids):
    """
    pycocotools 평가를 위해, validation set에 대한 Ground Truth(실제 정답)를
    COCO API가 요구하는 형식으로 변환합니다.
    """
    gt_images = []
    gt_annotations = []
    ann_id_counter = 1

    for img_id in val_ids:
        img_info = img_meta[img_id]
        gt_images.append({
            'id': img_id,
            'width': img_info['width'],
            'height': img_info['height'],
            'file_name': img_info['file_name']
        })

        for ann in ann_by_img.get(img_id, []):
            gt_annotations.append({
                'id': ann_id_counter,
                'image_id': img_id,
                'category_id': ann['category_id'],
                'bbox': ann['bbox'],
                'area': ann['bbox'][2] * ann['bbox'][3],
                'iscrowd': 0
            })
            ann_id_counter += 1

    return {
        # --- (수정) 아래 두 줄 추가 ---
        'info': {},      # info 키 추가 (내용은 비어도 됨)
        'licenses': [],  # licenses 키 추가 (내용은 비어도 됨)
        # --------------------------
        'images': gt_images,
        'annotations': gt_annotations,
        'categories': COCO['categories']
    }

@torch.no_grad()
def predict_for_eval(model, val_ids, device):
    """
    Validation set 이미지들에 대해 예측을 수행하고,
    pycocotools 형식에 맞는 결과 리스트를 반환합니다.
    """
    model.eval()
    predictions = []
    for img_id in val_ids:
        with Image.open(id2path[img_id]) as im:
            im = im.convert("RGB")
        img_tensor = to_tensor(im).to(device)

        outputs = model([img_tensor])[0]

        boxes = outputs['boxes'].cpu().numpy()
        labels = outputs['labels'].cpu().numpy() + 1 # 0-based -> 1-based
        scores = outputs['scores'].cpu().numpy()

        for box, label, score in zip(boxes, labels, scores):
            if score < EVAL_SCORE_THR:
                continue

            box[2] -= box[0] # x2,y2 -> w,h
            box[3] -= box[1]

            predictions.append({
                'image_id': img_id,
                'category_id': int(label),
                'bbox': box.tolist(),
                'score': float(score)
            })
    return predictions

def evaluate_map(model, val_ids, device):
    """mAP 평가를 총괄하는 메인 함수"""
    if not val_ids:
        print("[EVAL] 검증 데이터셋이 없어 mAP 평가를 건너뜁니다.")
        return None

    # 1. Ground Truth(정답) 데이터 준비
    gt_dataset = build_gt_for_eval(val_ids)
    coco_gt = COCOapi()
    coco_gt.dataset = gt_dataset
    coco_gt.createIndex()

    # 2. Prediction(모델 예측) 데이터 준비
    pred_results = predict_for_eval(model, val_ids, device)
    coco_dt = coco_gt.loadRes(pred_results)

    # 3. pycocotools를 이용해 mAP 계산 및 출력
    evaluator = COCOeval(coco_gt, coco_dt, iouType='bbox')
    evaluator.params.iouThrs = np.linspace(0.75, 0.95, 5) # 대회 평가지표
    evaluator.evaluate()
    evaluator.accumulate()
    evaluator.summarize()

    map_score = evaluator.stats[0]
    print(f"\n[RESULT] 최종 mAP @[IoU=0.75:0.95] = {map_score:.4f}")
    return map_score

def _parse_image_id_from_name(fname: str) -> int:
    """파일명에서 숫자 부분만 추출하는 헬퍼 함수"""
    return int("".join(filter(str.isdigit, Path(fname).stem)))

@torch.no_grad()
def export_submission_csv(model, test_dir, out_csv, device):
    """테스트 이미지에 대한 예측 결과를 submission.csv 파일로 저장"""
    if not test_dir.exists():
        print(f"[SUBMIT] 테스트 폴더({test_dir})가 없어 제출 파일 생성을 건너뜁니다.")
        return

    model.eval()
    results = []
    ann_id_counter = 1
    test_files = sorted(list(test_dir.glob("*.png")) + list(test_dir.glob("*.jpg")))

    for img_path in test_files:
        image_id = _parse_image_id_from_name(img_path.name)
        with Image.open(img_path) as im:
            im = im.convert("RGB")
        img_tensor = to_tensor(im).to(device)

        outputs = model([img_tensor])[0]

        boxes = outputs['boxes'].cpu().numpy()
        labels = outputs['labels'].cpu().numpy() + 1 # 0-based -> 1-based
        scores = outputs['scores'].cpu().numpy()

        for box, label, score in zip(boxes, labels, scores):
            if score < 0.05: continue

            results.append([
                ann_id_counter, image_id, int(label),
                box[0], box[1], box[2] - box[0], box[3] - box[1], # xyxy -> xywh
                score
            ])
            ann_id_counter += 1

    # CSV 파일 작성
    with open(out_csv, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["annotation_id", "image_id", "category_id", "bbox_x", "bbox_y", "bbox_w", "bbox_h", "score"])
        writer.writerows(results)
    print(f"\n[SUBMIT] 제출 파일 생성 완료: {out_csv}")
    print(f"  -> 총 {len(results)}개의 객체를 {len(test_files)}개의 이미지에서 예측했습니다.")

# --- 최종 실행 ---
print("\n--- mAP 평가 시작 ---")
stats = evaluate_map(model, val_ids, DEVICE)

print("\n--- 제출 파일 생성 시작 ---")
export_submission_csv(model, TEST_DIR, SUBMIT_CSV, DEVICE)


--- mAP 평가 시작 ---
creating index...
index created!
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.58s).
Accumulating evaluation results...
DONE (t=1.06s).
 Average Precision  (AP) @[ IoU=0.75:0.95 | area=   all | maxDets=100 ] = 0.325
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.341
 Average Precision  (AP) @[ IoU=0.75:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75:0.95 | area= large | maxDets=100 ] = 0.325
 Average Recall     (AR) @[ IoU=0.75:0.95 | area=   all | maxDets=  1 ] = 0.914
 Average Recall     (AR) @[ IoU=0.75:0.95 | area=   all | maxDets= 10 ] = 0.920
 Average Recall     (AR) @[ IoU=0.75:0.95 | area=   all | maxDets=100 ] = 0.920
 Average Reca

## 섹터 11: 결과 시각화

In [17]:
# ================================================================
# [섹터 11] 결과 시각화 (NameError 수정)
# ================================================================
# 폰트 설치 (Colab에서 한글 깨짐 방지, 이미 실행했다면 생략 가능)
!apt-get -qq install -y fonts-nanum

from IPython.display import display
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from torchvision.transforms.functional import to_tensor

# --- 시각화 설정값 ---
SCORE_THR = 0.25      # 시각화할 예측의 최소 점수(confidence)
TOPK_PER_IMAGE = 4    # 이미지당 최대 몇 개의 Bbox를 그릴지
N_VIS = 8             # 총 몇 개의 이미지를 시각화할지
VIS_DIR = ROOT / "vis"
VIS_DIR.mkdir(parents=True, exist_ok=True)

# --- (오류 수정) cat_id2name 변수 생성 ---
# 이 셀을 단독으로 실행할 경우를 대비하여, 'cat_id2name' 변수를 여기서 다시 정의합니다.
# 이 변수는 원래 [섹터 3]에서 생성됩니다.
cat_id2name = {c["id"]: c["name"] for c in COCO["categories"]}
# -----------------------------------------


# --- 함수 정의 ---
def get_korean_font(size=15):
    """Colab에 설치된 나눔고딕 폰트를 가져오는 함수"""
    font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
    try:
        return ImageFont.truetype(font_path, size)
    except IOError:
        print(f"[FONT] 나눔고딕 폰트를 찾을 수 없습니다. 기본 폰트를 사용합니다.")
        return ImageFont.load_default()

def pretty_name(category_id: int):
    """
    카테고리 ID를 입력받아, 가장 보기 좋은 형태의 알약 이름으로 변환합니다.
    (예: '기넥신에프정(은행엽엑스)(수출용)' -> '기넥신에프정')
    """
    if category_id in cat_id2name:
        # 괄호와 그 안의 내용을 제거하여 이름을 간소화
        name = cat_id2name[category_id]
        return name.split('(')[0]
    return f"CLS_{category_id}"

@torch.no_grad()
def visualize_samples(model, image_ids):
    """주어진 이미지 ID들에 대해 모델 예측을 시각화하는 메인 함수"""
    model.eval()
    font = get_korean_font()

    images_to_display = []

    for img_id in image_ids:
        path = id2path.get(img_id)
        if not path or not path.exists():
            continue

        # 1. 이미지 로드 및 추론 실행
        with Image.open(path) as im:
            im = im.convert("RGB")
        img_tensor = to_tensor(im).to(DEVICE)
        outputs = model([img_tensor])[0]

        # 2. 결과 필터링
        boxes = outputs['boxes'].cpu().numpy()
        labels = outputs['labels'].cpu().numpy() + 1 # 0-based -> 1-based
        scores = outputs['scores'].cpu().numpy()

        keep = scores >= SCORE_THR
        boxes, labels, scores = boxes[keep], labels[keep], scores[keep]

        if len(boxes) > TOPK_PER_IMAGE:
            indices = np.argsort(-scores)[:TOPK_PER_IMAGE]
            boxes, labels, scores = boxes[indices], labels[indices], scores[indices]

        # 3. PIL을 이용해 Bbox와 라벨 그리기
        vis_image = im.copy()
        draw = ImageDraw.Draw(vis_image)

        for box, label, score in zip(boxes, labels, scores):
            draw.rectangle(box.tolist(), outline="red", width=3)
            text = f"{pretty_name(label)} {score:.2f}"
            text_bbox = draw.textbbox((box[0], box[1] - 18), text, font=font)
            draw.rectangle(text_bbox, fill="red")
            draw.text((box[0], box[1] - 18), text, fill="white", font=font)

        # 4. 결과 저장 및 표시 리스트에 추가
        save_path = VIS_DIR / f"vis_{Path(path).name}"
        vis_image.save(save_path)

        if len(images_to_display) < 4:
             images_to_display.append(vis_image)

    print(f"\n[VISUALIZE] 시각화 결과 {len(image_ids)}개를 {VIS_DIR} 폴더에 저장했습니다.")

    for img in images_to_display:
        display(img)

# --- 최종 실행 ---
if val_ids:
    sample_ids = random.sample(val_ids, min(N_VIS, len(val_ids)))
    print(f"{len(sample_ids)}개의 검증 이미지에 대한 예측을 시각화합니다...")
    visualize_samples(model, sample_ids)
else:
    print("시각화할 검증 이미지가 없습니다.")

Output hidden; open in https://colab.research.google.com to view.